In [293]:
# TODO: load captions
# TODO: load perturbations
# TODO: generate perturbed captions
# TODO: update dataset root in config files

In [294]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [295]:
import sys

path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/'

if path not in sys.path:
    sys.path.append(path)


In [296]:
from src.utils.utils import get_config
from munch import Munch

dataset = 'f30k_aug'
dataset = 'f30k'
model = 'align'
task = 't2i'
perturbation = 'all_ordered'

# config = get_config(dataset=dataset, model=model)

path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/config/align/f30k/development_local.yaml'

with open(path, 'rb') as f:
    config = Munch.fromYAML(f)


config.args = Munch(
    dataset=dataset,
    model=model, 
    task=task,
    perturbation=perturbation,
    compute_from_scratch=False
    )


# TODO: collate_fn -> raw_caption, caption_emb, img_emb
# TODO: load precomputed embeddings

from src.utils.utils import get_config, get_logger, get_model

model = get_model(config)

logging = get_logger(config=config)
config.logging = logging

KeyboardInterrupt: 

In [ ]:
# T2I task
img_filenames, img_emb = model.compute_image_embeddings(compute_from_scratch=False)


Loaded embeddings from: /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/precomputed/align/f30k-img-embeddings.pkl
Image embeddigns are already precomputed
Loaded precomputed filenames and embeddings from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/precomputed/align/f30k-img-embeddings.pkl


In [345]:
from src.utils.dataset_preprocessing import load_json_annotations

json_file = load_json_annotations(config)

json_file_aug = load_json_annotations(config, augmented=True)

file_path:  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k.json
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k.json
file_path:  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json


In [ ]:
import pickle

path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/results/f30k/align/t2i/char_swap-results.pkl'

with open(path, 'rb') as f:
    data = pickle.load(f)

In [332]:
from src.data.dataset import Dataset

ds_split_f30k_aug = Dataset(config=config, split='test', json_file=json_file)

file_path:  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json


In [ ]:
len(json_file_aug.keys())

1000

In [ ]:
json_file['images'][0].keys()

dict_keys(['sentids', 'imgid', 'sentences', 'split', 'filename'])

In [ ]:
json_file['images'][25]['sentences'][0]['raw']

'The man with pierced ears is wearing glasses and an orange hat.'

In [ ]:
json_file['images'][25]['sentences'][1]['raw']

'A man with glasses is wearing a beer can crocheted hat.'

In [ ]:
json_file['images'][25]['sentences'][2]['raw']

'A man with gauges and glasses is wearing a Blitz hat.'

In [300]:
from src.data.dataset import Dataset

ds_split_f30k = Dataset(config=config, split='test', json_file=json_file)

file_path:  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/annotations/dataset_flickr30k_aug.json


In [301]:
ds_split_f30k[1][0]

'false true true is up it false up false false is up and and it up false false up false is it it is and true false and and true true up false is and and it is false and and it false and and and true and up false false is it it is false false it false and up and false true and it is it and and true it up it it and true false it true and false is rreererrerererrrererererererererereereerereererererereerererererererererererer false false is true true false true is and a true it is false true it and and it and and true and and is true it and false is true false true false up is it true is true it it and and true is true true it is true up adenine false and up false is is false true false and is true true and true and and true true true false false true is and and up is is and it and false up false and and is false and is up up is nicotinamide is and true it false up false true it and true is it false it is and false up and true false and is true true is true and false false it up true is is

In [ ]:
ALL_PERTURBATIONS  = [
    'char_swap',
    'missing_char',
    'extra_char',
    'nearby_char',
    # 'probability_based_letter_change',
    'synonym_noun',
    'synonym_adj',
    'distraction_true',
    'distraction_false',
    'shuffle_nouns_and_adj',
    'shuffle_all_words',
    'shuffle_allbut_nouns_and_adj',
    'shuffle_within_trigrams',
    'shuffle_trigrams'
]

In [ ]:
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 


def split_list_into_k_sublists(l, k):
    """
    l: List
    k: number of perturbations
    """
    # initial sublist size
    sublist_size = len(l) // k
    # print('sublist_size: ', sublist_size)

    if len(l) % k > 0:
        sublist_size += 1

    ans = list(divide_chunks(l, sublist_size))

    assert len(ans) == k

    return ans

# get k img_id sublists

k_img_ids_sublists = split_list_into_k_sublists(l=img_ids, k=len(ALL_PERTURBATIONS))



In [336]:
ds_split_f30k.images.keys()

dict_keys([25, 34, 51, 89, 97, 124, 133, 134, 137, 199, 219, 222, 228, 246, 259, 279, 304, 318, 340, 395, 434, 472, 487, 520, 539, 565, 615, 654, 661, 670, 734, 748, 754, 760, 828, 856, 887, 906, 917, 972, 973, 1023, 1030, 1044, 1072, 1117, 1196, 1202, 1206, 1284, 1305, 1306, 1310, 1343, 1358, 1390, 1401, 1417, 1480, 1576, 1594, 1610, 1646, 1723, 1771, 1821, 1833, 1895, 1981, 1983, 1984, 1987, 2044, 2054, 2103, 2112, 2113, 2124, 2130, 2131, 2139, 2161, 2180, 2197, 2260, 2265, 2319, 2330, 2410, 2412, 2429, 2433, 2458, 2523, 2551, 2572, 2600, 2615, 2678, 2690, 2788, 2803, 2848, 2851, 2906, 2909, 2955, 2956, 3008, 3078, 3082, 3087, 3097, 3109, 3130, 3162, 3245, 3412, 3437, 3451, 3459, 3476, 3500, 3539, 3648, 3701, 3703, 3724, 3771, 3774, 3826, 3834, 3951, 3972, 4007, 4027, 4028, 4041, 4313, 4317, 4343, 4367, 4402, 4452, 4501, 4510, 4556, 4562, 4663, 4686, 4748, 4801, 4816, 4879, 4881, 4882, 4902, 4942, 4981, 5016, 5056, 5160, 5171, 5240, 5261, 5327, 5330, 5343, 5441, 5541, 5580, 5581, 559

In [ ]:
from src.perturbations.perturbation import Perturbation

# initialize perturbation
perturbation_char_swap = Perturbation(
    config=config,
    perturbation_type='char_swap')

In [320]:
from tqdm import tqdm
from src.perturbations.perturbation import Perturbation

for perturbation_type, sublist in zip(ALL_PERTURBATIONS, k_img_ids_sublists):
    print(perturbation_type)
    # initialize perturbation
    perturbation = Perturbation(
        config=config,
        perturbation_type=perturbation_type)
    
    for img_id in tqdm(sublist):
        sentids = ds_split_f30k.images[img_id]['sentids']
        for sentid in sentids:
            print(sentid)
            tmp_capt = ds_split_f30k.captions[sentid]['raw']
            print('Original caption: ', tmp_capt)
            perturbed_caption = perturbation.apply_perturbation_to_caption(tmp_capt)
            ds_split_f30k.update_caption(caption_idx=sentid, new_caption=perturbed_caption)
            print('Modified caption: ', perturbed_caption)


    

char_swap
perturbation_type:  char_swap
Initialized perturation:  char_swap


100%|██████████| 77/77 [00:00<00:00, 32948.52it/s]


125
Original caption:  The man with pierced ears is wearing glasses and an orange hat.
Modified caption:  The man with iperced ears is wearing glasses and an orange hat.
170
Original caption:  A black and white dog is running in a grassy garden surrounded by a white fence.
Modified caption:  A black and white dog is running in a grassy garden surrounded by a hwite fence.
255
Original caption:  A young female student performing a downward kick to break a board held by her Karate instructor.
Modified caption:  A young female student performing a downward kick to break a baord held by her Karate instructor.
445
Original caption:  Five snowmobile riders all wearing helmets and goggles line up in a snowy clearing in a forest in front of their snowmobiles; they are all wearing black snow pants and from left to right they are wearing a black coat, white coat, red coat, blue coat, and black coat.
Modified caption:  Five snowmobile riders all wearing helemts and goggles line up in a snowy clear

100%|██████████| 77/77 [00:00<00:00, 38498.20it/s]


10620
Original caption:  A brown dog walks in the grass with its tongue hanging out.
Modified caption:  A brown dog walks in the grass with its tongue haning out.
10650
Original caption:  Two men sitting in front of a building and a field of grass in front of them.
Modified caption:  Two men sitting in front of a building and a field f grass in front of them.
10655
Original caption:  Several people, including a shirtless man and a woman in purple shorts which say "P.I.N.K." on the back, are walking through a crowded outdoor area.
Modified caption:  Several people, including a shirtless man and a woman in purple shorts which say "P.I.N.K." on the back, ar walking through a crowded outdoor area.
10695
Original caption:  Two boys try to attach fruits to a bicycle on a dirt road.
Modified caption:  Two boys try to attach fruits to a bicycle on a dir road.
10805
Original caption:  An African American man with a black hat and black tank top plays the drums on some buckets for donations.
Modi

...

Initialized perturation:  shuffle_within_trigrams


100%|██████████| 77/77 [00:00<00:00, 16071.73it/s]


130195
Original caption:  A female gymnast, captured in midair during a jump.
Modified caption:  A female gymnast captured in , during a midair . jump
130245
Original caption:  A man in an orange visor crossing the street with a ice cream vendor in the background and a woman about to cross the street as well.
Modified caption:  A man in orange visor an the street crossing a ice with vendor cream in the background and about woman a to cross the as well street .
130250
Original caption:  A man in a long gray gown and hat is standing with his hand on a mule that is wearing blankets and decorations on a stone street outside of a building, with a number of other people in the background.
Modified caption:  A man in a gray long and hat gown with is standing on hand his that mule a is blankets wearing on decorations and stone a street a outside of building , with a number of people other in background the .
130420
Original caption:  Two young children working together to operate a bicycle.
Mo

100%|██████████| 76/76 [00:00<00:00, 18709.19it/s]

142400
Original caption:  A woman with short red-hair is in some very cloudy water by a shoreline and only her head is above the water.
Modified caption:  a shoreline and water . is above the short red-hair is in some very cloudy water by A woman with only her head
142850
Original caption:  Two young boys sitting on a sunlit floor smiling and holding a black lab puppy.
Modified caption:  sunlit floor smiling black lab puppy and holding a . sitting on a Two young boys
142910
Original caption:  Several children are practicing martial arts moves in a room with a red floor.
Modified caption:  room with a Several children are moves in a red floor . practicing martial arts
143025
Original caption:  A man tired on a couch just waiting to go home from work.
Modified caption:  go home from work . just waiting to on a couch A man tired
143305
Original caption:  A lady in a gray sweater has a conversation with three other people at the same table.
Modified caption:  with three other A lady in peo

In [325]:
torch.save(ds_split_f30k, '../deliverables/datasets/custom_dataset.pth')

In [326]:
tmp_ds = torch.load('../deliverables/datasets/custom_dataset.pth')

In [327]:
tmp_ds.captions[125]['raw']

'The man with iperced ears is wearing glasses and an orange hat.'

In [324]:
ds_split_f30k.captions[125]['raw']

'The man with iperced ears is wearing glasses and an orange hat.'

In [ ]:
from tqdm import tqdm

for img_id in tqdm(k_img_ids_sublists[0]):
    try:
        # filename = ds_split_f30k.images[key]['filename'].split('.')[0]
        sentids = ds_split_f30k.images[key]['sentids']
        # print(img_id)
        # print('filename: ', filename)
        # get caption, assign new value
        for sentid in sentids:
            # print(sentid)
            tmp_capt = ds_split_f30k.captions[sentid]['raw']
            # print(tmp_capt)
            perturbed_caption = perturbation_char_swap.apply_perturbation_to_caption(tmp_capt)
            ds_split_f30k.update_caption(caption_idx=sentid, new_caption=perturbed_caption)
            # print(perturbed_caption)

        # perturbation_char_swap.apply_perturbation_to_caption(caption)
    except Exception as e:
        print('problem with caption: ', caption)
        print(e)

100%|██████████| 77/77 [00:00<00:00, 86.67it/s]
